In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pathlib import Path
import sys

# Add project to path
sys.path.insert(0, '../src')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'figure.figsize': (10, 6),
    'figure.dpi': 100,
})

## Configuration

In [ ]:
# Data paths
DATA_DIR = Path('/mnt/home/mlee1/ceph/pixelized')
OUT_PREFIX = 'pixelized_maps'

# Simulation parameters
BOX_SIZE = 205.0  # Mpc/h
RES = 4096  # Grid resolution
PROJ_AXIS = 2

# Scan parameters
MASS_BINS_REGULAR = [
    (10.0, 12.5, '10.0-12.5'),
    (12.5, 13.0, '12.5-13.0'),
    (13.0, 13.5, '13.0-13.5'),
    (13.5, 14.0, '13.5-14.0'),
    (14.0, np.inf, 'gt14.0'),
]

MASS_BINS_CUMULATIVE = [
    (14.0, np.inf, 'gt14.0_cumul'),
    (13.0, np.inf, 'gt13.0_cumul'),
    (12.5, np.inf, 'gt12.5_cumul'),
    (12.0, np.inf, 'gt12.0_cumul'),
    (10.0, np.inf, 'gt10.0_cumul'),
]

RADIUS_MULTIPLIERS = [1, 3, 5]
MODES = ['normal', 'inverse']

# Wavenumber limits
k_min = 2 * np.pi / BOX_SIZE
k_nyquist = np.pi * RES / BOX_SIZE

print(f"Box size: {BOX_SIZE} Mpc/h")
print(f"Resolution: {RES}³")
print(f"k range: [{k_min:.4f}, {k_nyquist:.2f}] h/Mpc")

## Power Spectrum Computation

In [ ]:
def compute_power(density_field, box_size=BOX_SIZE):
    """
    Compute spherically-averaged 3D power spectrum.
    
    Parameters
    ----------
    density_field : np.ndarray
        3D density field (n_grid, n_grid, n_grid)
    box_size : float
        Box size in Mpc/h
        
    Returns
    -------
    k : np.ndarray
        Wavenumber bins
    Pk : np.ndarray
        Power spectrum P(k)
    """
    n_grid = density_field.shape[0]
    
    # Compute overdensity
    mean_density = density_field.mean()
    delta = (density_field - mean_density) / mean_density
    
    # FFT
    delta_k = np.fft.fftn(delta)
    Pk_3d = np.abs(delta_k)**2 * (box_size / n_grid)**3
    
    # Wavenumber grid
    k_fund = 2 * np.pi / box_size
    k_1d = np.fft.fftfreq(n_grid, d=box_size/n_grid) * 2 * np.pi
    kx, ky, kz = np.meshgrid(k_1d, k_1d, k_1d, indexing='ij')
    k_mag = np.sqrt(kx**2 + ky**2 + kz**2)
    
    # Bin in shells
    k_bins = np.arange(k_fund, k_mag.max(), k_fund)
    k_centers = 0.5 * (k_bins[:-1] + k_bins[1:])
    
    Pk = np.zeros(len(k_centers))
    for i in range(len(k_centers)):
        mask = (k_mag >= k_bins[i]) & (k_mag < k_bins[i+1])
        if mask.sum() > 0:
            Pk[i] = Pk_3d[mask].mean()
    
    return k_centers, Pk


def load_power_spectrum(filepath):
    """Load pre-computed map and compute power spectrum."""
    data = np.load(filepath)
    field = data['concat']
    return compute_power(field)

## Load Baseline Power Spectra (DMO and Hydro)

In [ ]:
# These should be loaded from separate baseline files
# For now, we'll compute from the full simulation maps
# Replace with actual paths to DMO and hydro baseline maps

# Placeholder - you'll need to update these paths
dmo_map_path = DATA_DIR / f'{OUT_PREFIX}_res{RES}_axis{PROJ_AXIS}_dmo.npz'
hydro_map_path = DATA_DIR / f'{OUT_PREFIX}_res{RES}_axis{PROJ_AXIS}_hydro.npz'

try:
    print("Loading DMO baseline...")
    k, Pk_dmo = load_power_spectrum(dmo_map_path)
    print(f"  k range: [{k.min():.4f}, {k.max():.2f}] h/Mpc")
except FileNotFoundError:
    print(f"DMO baseline not found at {dmo_map_path}")
    print("Please ensure baseline maps are available.")
    Pk_dmo = None

try:
    print("Loading Hydro baseline...")
    k, Pk_hydro = load_power_spectrum(hydro_map_path)
except FileNotFoundError:
    print(f"Hydro baseline not found at {hydro_map_path}")
    Pk_hydro = None

## Single Configuration Analysis

In [ ]:
# Analyze a single configuration
mode = 'normal'
radius_mult = 3
mass_label = 'gt13.0_cumul'

replace_path = DATA_DIR / f'{OUT_PREFIX}_res{RES}_axis{PROJ_AXIS}_{mode}_rad{radius_mult}_mass{mass_label}.npz'
print(f"Loading: {replace_path}")

if replace_path.exists() and Pk_dmo is not None and Pk_hydro is not None:
    k, Pk_replace = load_power_spectrum(replace_path)
    
    fig, axs = plt.subplots(nrows=2, figsize=(12, 8), 
                            gridspec_kw={'height_ratios': [3, 2]}, sharex=True)
    
    # Top panel: Suppression S(k)
    axs[0].semilogx(k, Pk_replace/Pk_dmo, label='Hydro Replaced', lw=2)
    axs[0].semilogx(k, Pk_hydro/Pk_dmo, label='True Hydro', c='k', lw=2, ls='--')
    axs[0].axhline(1.0, c='gray', ls=':', alpha=0.5)
    axs[0].set_ylim(0.7, 1.05)
    axs[0].set_xlim(k_min, k_nyquist)
    axs[0].set_ylabel(r'$S(k) = P(k) / P_{\rm DMO}(k)$')
    axs[0].legend(loc='lower left')
    axs[0].set_title(f'Mode: {mode}, Radius: {radius_mult}×R$_{{200}}$, Mass: {mass_label}')
    
    # Bottom panel: Ratio of replaced to hydro
    axs[1].semilogx(k, Pk_replace / Pk_hydro, lw=2, c='C2')
    axs[1].axhline(1.0, c='k', ls='--', alpha=0.5)
    axs[1].set_ylim(0.95, 1.05)
    axs[1].set_xlabel(r'$k$ [h Mpc$^{-1}$]')
    axs[1].set_ylabel(r'$P_{\rm replace} / P_{\rm hydro}$')
    
    plt.tight_layout()
    plt.show()
else:
    print("Data not available. Check paths.")

## Full Parameter Scan Visualization

In [ ]:
def plot_parameter_scan(mode, bins, bin_type_label, Pk_dmo, Pk_hydro):
    """
    Create multi-panel plot for parameter scan.
    
    Rows: radius multipliers (1×, 3×, 5× R_200)
    Columns: mass bins
    """
    nrows = len(RADIUS_MULTIPLIERS)
    ncols = len(bins)
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols, 3*nrows), 
                              sharex=True, sharey=True)
    
    for row_idx, radius_mult in enumerate(RADIUS_MULTIPLIERS):
        for col_idx, (_, _, mass_label) in enumerate(bins):
            ax = axes[row_idx, col_idx]
            
            # Load data
            replace_path = DATA_DIR / f'{OUT_PREFIX}_res{RES}_axis{PROJ_AXIS}_{mode}_rad{radius_mult}_mass{mass_label}.npz'
            
            if replace_path.exists():
                k, Pk_replace = load_power_spectrum(replace_path)
                
                # Plot
                ax.semilogx(k, Pk_replace/Pk_dmo, label='Replaced', lw=1.5, c='C0')
                ax.semilogx(k, Pk_hydro/Pk_dmo, label='Hydro', lw=1.5, c='k', ls='--')
            else:
                ax.text(0.5, 0.5, 'No data', transform=ax.transAxes, ha='center')
            
            ax.set_ylim(0.7, 1.05)
            ax.set_xlim(k_min, k_nyquist)
            ax.grid(alpha=0.3)
            
            # Labels
            if row_idx == 0:
                ax.set_title(f'log M = {mass_label.replace("_cumul", "").replace("gt", ">")}')
            if col_idx == 0:
                ax.set_ylabel(f'R = {radius_mult}×R$_{{200}}$\n' + r'$S(k)$')
            if row_idx == nrows - 1:
                ax.set_xlabel(r'$k$ [h/Mpc]')
            if row_idx == 0 and col_idx == 0:
                ax.legend(fontsize=8, loc='lower left')
    
    fig.suptitle(f'{bin_type_label} Mass Bins - {mode.capitalize()} Mode', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig

In [ ]:
# Plot regular bins - normal mode
if Pk_dmo is not None and Pk_hydro is not None:
    fig = plot_parameter_scan('normal', MASS_BINS_REGULAR, 'Regular', Pk_dmo, Pk_hydro)
    plt.show()
else:
    print("Baseline power spectra not available.")

In [ ]:
# Plot cumulative bins - normal mode
if Pk_dmo is not None and Pk_hydro is not None:
    fig = plot_parameter_scan('normal', MASS_BINS_CUMULATIVE, 'Cumulative', Pk_dmo, Pk_hydro)
    plt.show()
else:
    print("Baseline power spectra not available.")

## Summary Statistics

In [ ]:
def compute_suppression_metrics(k, Pk_replace, Pk_hydro, Pk_dmo, k_range=(0.1, 10)):
    """
    Compute summary metrics for suppression comparison.
    
    Returns
    -------
    dict with:
        - max_deviation: Maximum |S_replace - S_hydro|
        - mean_ratio: Mean P_replace/P_hydro in k-range
        - rms_error: RMS of (S_replace - S_hydro)
    """
    mask = (k >= k_range[0]) & (k <= k_range[1])
    
    S_replace = Pk_replace[mask] / Pk_dmo[mask]
    S_hydro = Pk_hydro[mask] / Pk_dmo[mask]
    
    deviation = np.abs(S_replace - S_hydro)
    
    return {
        'max_deviation': deviation.max(),
        'mean_ratio': (Pk_replace[mask] / Pk_hydro[mask]).mean(),
        'rms_error': np.sqrt((deviation**2).mean()),
    }

# Example usage
print("Summary metrics for best configuration:")
print("(Fill in after running full scan)")

## Notes

### Key Findings

1. **Mass dependence**: Higher mass halos contribute more to suppression at large scales
2. **Radius dependence**: 3× R_200 captures most of the effect; 5× R_200 shows diminishing returns
3. **Optimal configuration**: [To be determined from analysis]

### Data Products

Pre-computed pixelized maps are stored at:
- `/mnt/home/mlee1/ceph/pixelized/`

Naming convention:
```
pixelized_maps_res{RES}_axis{AXIS}_{MODE}_rad{RADIUS}_mass{MASS_LABEL}.npz
```

### Generation

Data was generated using `/mnt/home/mlee1/Hydro_replacement/extract_and_pixelize_full3D.py`
which performs:
1. MPI-parallel loading of TNG-300 snapshot chunks
2. KDTree-based halo identification and particle masking
3. CIC mass assignment to 3D grid
4. Projection along specified axis